In [31]:
import numpy as np
from scipy.linalg import ishermitian
import matplotlib.pyplot as plt
import random
from fem_1d import *
from qva import *
from pauliDecomp import *
from scipy.sparse import diags
import pprint
pp = pprint.PrettyPrinter(width=110, compact=True)
import qiskit.quantum_info as qi
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
#from qiskit_aer.primitives import Estimator  # import change!!!
from qiskit.primitives import Estimator

In [32]:


###############################
#     <0| U^d An V(k) |0>     #
###############################
   # b_psi(parameters,g):
nqbits = 2
nlayers = 2
ni = 2**(nqbits)
nx = ni + 2
g = ["XX"]

qubits = [0,1,2,3]
for i in range(0,nqbits):
     qubits.append(i)

    ## Getting the U operator ##

uc = QuantumCircuit(nqbits)
uc.h(0)
uc.h(1)
U = qi.Operator(uc)
print("uc: ",uc)
print("U: ",U)
print("U dagger: ",U.adjoint())
print("Is the U-Operator Unitary? ",U.is_unitary())
print("Is the U-Operator Hermition? ",ishermitian(U.data))

    ## Getting the V(k) operator ##

Vcirc = QuantumCircuit(nqbits)
parameters = [1, 2, 3, 4]
Vcirc = ansatz_RYZ(Vcirc, qubits, parameters, nlayers)
print(Vcirc)
V = qi.Operator(Vcirc)
print(Vcirc)
print("V: ",V)
print("Is the V-Operator Unitary? ",V.is_unitary())
print("Is the V-Operator Hermition? ",ishermitian(V.data))

    ## Getting the An operator ##

    #coeffs = c
labels = g
An = SparsePauliOp(labels)
An = qi.Operator(An)
print("An: ",An)
print("Is the An-Operator Unitary? ",An.is_unitary())
print("Is the An-Operator Hermition? ",ishermitian(An.data))

    ## Making the operator ##

op = V.compose(An.compose(U.adjoint()))  # <-- our circuit matches this order
print("Operator: ",op)
print("Is the Operator Unitary? ",op.is_unitary())
print("Is the Operator Hermition? ",ishermitian(op.data))
print("An: ",An)

    # NOTE: if op is hermitian, then it's expectation values should be all real

    ## prepare |0> state ##
state = QuantumCircuit(nqbits)
print("\n STATE \n")
print(state)

    ## Run estimator ##

estimator = Estimator()
expectation_value = estimator.run(state, op).result().values
    ##expectation_value = estimator.run(state, op, shots=10000).result().values
print("*********************************************")
print("USING OPERATORS")
print("REAL[ E[<0| U^d An V(k) |0>] ] = ",expectation_value[0].real)
print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",expectation_value[0].imag)
print("*********************************************")


#return expectation_value
    ###########################################################################
    ###########################################################################

    ## import old code ##

from qva import *

    ## establish gate and left hand side ##
g = [1,1]
circ_RHS = QuantumCircuit(nqbits+1)
circ_RHS.ch(0,1)
circ_RHS.ch(0,2)


print("*********************************************")
print("USING OUR OLD CODE")
    ##print("REAL[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Re",parameters,nlayers,circ_RHS))
    ##print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Im",parameters,nlayers,circ_RHS))

treal = bpsi_hadamard(nqbits,g,"Re",parameters,nlayers,circ_RHS)
timag = bpsi_hadamard(nqbits,g,"Im",parameters,nlayers,circ_RHS)
print("REAL[ E[<0| U^d An V(k) |0>] ] = ",treal)
print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",timag)
print("*********************************************")

uc:       ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘
U:  Operator([[ 0.5+0.j,  0.5+0.j,  0.5+0.j,  0.5+0.j],
          [ 0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j],
          [ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
          [ 0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
U dagger:  Operator([[ 0.5-0.j,  0.5-0.j,  0.5-0.j,  0.5-0.j],
          [ 0.5-0.j, -0.5-0.j,  0.5-0.j, -0.5-0.j],
          [ 0.5-0.j,  0.5-0.j, -0.5-0.j, -0.5-0.j],
          [ 0.5-0.j, -0.5-0.j, -0.5-0.j,  0.5-0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Is the U-Operator Unitary?  True
Is the U-Operator Hermition?  True
     ┌───────┐ ░    ┌───────┐ ░ 
q_0: ┤ Ry(1) ├─░──■─┤ Ry(3) ├─░─
     ├───────┤ ░  │ ├───────┤ ░ 
q_1: ┤ Ry(2) ├─░──■─┤ Ry(4) ├─░─
     └───────┘ ░    └───────┘ ░ 
     ┌───────┐ ░    ┌───────┐ ░ 
q_0: ┤ Ry(1) ├─░──■─┤ Ry(3) ├─░─
     ├───────┤ ░  │ ├───────┤ ░ 
q_1: ┤ Ry(2) ├─░──■─┤ Ry(4) ├─░─
     └───────┘ ░    └───────┘ ░ 
V:  Operator(

In [33]:
###############################
# <0| V(k)^D An^D Am V(k) |0> #
###############################

nqbits = 2
nlayers = 2

state = QuantumCircuit(nqbits)
parameters = [1, 2, 3, 4]
state = ansatz_RYZ(state, qubits, parameters, nlayers)
print(state)


An = SparsePauliOp.from_list([("YY", 1)])
An = An.adjoint()
An = qi.Operator(An)
print("An: ",An)
Am = SparsePauliOp.from_list([("XX", 1)])
AM = qi.Operator(Am)
print("Am: ", Am)

op = An.compose(Am)
print(op)

estimator = Estimator()
expectation_value = estimator.run(state, op).result().values
print(expectation_value)
##expectation_value = estimator.run(state, op, shots=10000).result().values
print("*********************************************")
print("USING OPERATORS")
print("REAL[ E[<0| V(k)^D Am^D An V(k) |0>] ] = ",expectation_value[0].real)
print("IMAG[ E[<0| V(k)^D Am^D An V(k) |0>] ] = ",expectation_value[0].imag)
print("*********************************************")

## odd code
g = [2,2]
k = [1,1]
print("*********************************************")
print("USING OUR OLD CODE")
##print("REAL[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Re",parameters,nlayers,circ_RHS))
##print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Im",parameters,nlayers,circ_RHS))

treal = psi_norm_hadamard(nqbits,g,k,"Re",parameters,nlayers)
timag = psi_norm_hadamard(nqbits,g,k,"Im",parameters,nlayers)
print("REAL[ E[<0| U^d An V(k) |0>] ] = ",treal)
print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",timag)
print("*********************************************")

     ┌───────┐ ░    ┌───────┐ ░ 
q_0: ┤ Ry(1) ├─░──■─┤ Ry(3) ├─░─
     ├───────┤ ░  │ ├───────┤ ░ 
q_1: ┤ Ry(2) ├─░──■─┤ Ry(4) ├─░─
     └───────┘ ░    └───────┘ ░ 
An:  Operator([[ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j],
          [ 0.+0.j,  0.+0.j,  1.-0.j,  0.+0.j],
          [ 0.+0.j,  1.-0.j,  0.+0.j,  0.+0.j],
          [-1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Am:  SparsePauliOp(['XX'],
              coeffs=[1.+0.j])
Operator([[-1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
[0.74915046]
*********************************************
USING OPERATORS
REAL[ E[<0| V(k)^D Am^D An V(k) |0>] ] =  0.7491504573141078
IMAG[ E[<0| V(k)^D Am^D An V(k) |0>] ] =  0.0
*********************************************
*********************************************
USING OUR OLD CO

In [43]:
###############################
#     <0| U^d An V(k) |0>     #
###############################
def b_psi(parameters,g):
    nqbits = 2
    nlayers = 2
    ni = 2**(nqbits)
    nx = ni + 2

    qubits = [0,1,2,3]
    for i in range(0,nqbits):
        qubits.append(i)

    ## Getting the U operator ##

    uc = QuantumCircuit(nqbits)
    uc.h(0)
    uc.h(1)
    U = qi.Operator(uc)
  
    ## Getting the V(k) operator ##

    Vcirc = QuantumCircuit(nqbits)
    ## parameters = [1, 2, 3, 4]
    Vcirc = ansatz_RYZ(Vcirc, qubits, parameters, nlayers)
    V = qi.Operator(Vcirc)

    ## Getting the An operator ##

    #coeffs = c
    labels = g
    An = SparsePauliOp(labels)
    An = qi.Operator(An)
 
    ## Making the operator ##

    op = V.compose(An.compose(U.adjoint()))  # <-- our circuit matches this order

    # NOTE: if op is hermitian, then it's expectation values should be all real

    ## prepare |0> state ##
    state = QuantumCircuit(nqbits)

    ## Run estimator ##

    estimator = Estimator()
    expectation_value = estimator.run(state, op).result().values
    ##expectation_value = estimator.run(state, op, shots=10000).result().values
    #print("*********************************************")
    #print("USING OPERATORS")
    #print("REAL[ E[<0| U^d An V(k) |0>] ] = ",expectation_value[0].real)
    #print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",expectation_value[0].imag)
    #print("*********************************************")


    return expectation_value
    ###########################################################################
    ###########################################################################



###############################
# <0| V(k)^D An^D Am V(k) |0> #
###############################

def psi_psi(parameters, g1, g2):
    nqbits = 2
    nlayers = 2

    state = QuantumCircuit(nqbits)
    state = ansatz_RYZ(state, qubits, parameters, nlayers)

    An = SparsePauliOp.from_list([(g1, 1)])
    An = An.adjoint()
    An = qi.Operator(An)

    Am = SparsePauliOp.from_list([(g2, 1)])
    AM = qi.Operator(Am)

    op = An.compose(Am)

    estimator = Estimator()
    expectation_value = estimator.run(state, op).result().values
    ##expectation_value = estimator.run(state, op, shots=10000).result().values
    #print("*********************************************")
    #print("USING OPERATORS")
    #print("REAL[ E[<0| V(k)^D Am^D An V(k) |0>] ] = ",expectation_value[0].real)
    #print("IMAG[ E[<0| V(k)^D Am^D An V(k) |0>] ] = ",expectation_value[0].imag)
    #print("*********************************************")


    return expectation_value

In [44]:
cost_values = []
my_gate_set = ["II","IX","XX","YY"]
my_coefficient_set = [2,-1,-0.5,-0.5]

In [46]:
norm = complex(0,0)
cost = complex(0,0)
for i in range(0,len(my_gate_set)):
    for j in range(0,len(my_gate_set)):
        norm += my_coefficient_set[i] * my_coefficient_set[j].conjugate() * complex(psi_psi(parameters, my_gate_set[i],my_gate_set[j])[0].real, psi_psi(parameters, my_gate_set[i],my_gate_set[j])[0].imag)
        t1 = complex(b_psi(parameters, my_gate_set[i])[0].real, b_psi(parameters, my_gate_set[i])[0].imag)
        t2 = complex(b_psi(parameters, my_gate_set[j])[0].real, b_psi(parameters, my_gate_set[j])[0].imag)
        cost +=  my_coefficient_set[i] * my_coefficient_set[j] * t1 * t2
cost = complex(cost)
norm = complex(norm)

result = 1-float(cost.real/norm.real)
cost_values.append(result)
print("iteration: ",len(cost_values)," || cost: ",result) #," || w: ",parameters)


iteration:  1  || cost:  0.9953251107320805


In [47]:
def run_qva(nmaxiter,c,g,b,parameters,method,rhobeg,ul,ur,uscale,reduced):
    nqbits = 2
    nlayers = 2
    n = 2**nqbits
    nparameters = nqbits + 2*(nqbits-1)*(nlayers-1)
    assert(nparameters == len(parameters))
    
    aux = 0
    qubitsRHS = [1]
    for i in range(1,nqbits):
        qubitsRHS.append(i+1)

    # Normalize this vector
    b = b/np.linalg.norm(b)
    #b = np.array(b,dtype=complex)
    
    # Get a circuit that takes 0 state and gives the vector
    automated_circ = QuantumCircuit(nqbits+1)
    automated_circ = obtain_circuit_from_vec(b,automated_circ,qubitsRHS)
    circ_RHS = QuantumCircuit(nqbits+1)
    circ_RHS = control_version(automated_circ,circ_RHS,aux)
    
    nit = 0
    cost_values = []
    out = minimize(cost_function, parameters, args=(nqbits, g, c, nlayers, circ_RHS, cost_values, nit), method=method, options={'maxiter':maxiter,'rhobeg':rhobeg,'disp':False}) # Works great for 3 qubit
    #print("cost values: ",cost_values)
    print(out)
    final_parameters = out['x'][0:len(parameters)]
    parameters=final_parameters
    
    qubits = []
    for i in range(0,nqbits):
        qubits.append(i)
    
    circ = QuantumCircuit(nqbits, nqbits)
    circ = ansatz_RYZ(circ, qubits, final_parameters, nlayers)
    circ.save_statevector()
    backend = Aer.get_backend('aer_simulator')
    t_circ = transpile(circ, backend)
    qobj = assemble(t_circ)
    job = backend.run(qobj)

    result = job.result()
    o = result.get_statevector(circ, decimals=3)
    print("RESULT 3: {o}")
    
    u_internal = np.absolute(np.real(o))
    u_internal = u_internal * (uscale/u_internal[0])
    
    n = n + 2
    u = np.zeros(n)
    if reduced == 0:
        u = u_internal
    else:
        u[0] = ul
        u[n-1] = ur
        u[1:n-1] = u_internal
        
    #print("cost_Values: ".cost_values)
    return u,parameters,cost_values